In [88]:
# Import libraries
from asyncio import events
import pandas as pd
from redcap import Project
wd = "/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Erin Cobry/Summer vs. School Sleep/"
# Setup REDCap import
keys = pd.read_csv("/Users/timvigers/Documents/Work/api_keys.csv")
api_key = keys["API Key"].loc[keys["Study"] ==
                              "Advanced Hybrid Closed-Loop and Sleep"][0]
api_url = "https://redcap.ucdenver.edu/api/"
project = Project(api_url, api_key)
# Visits per Erin and Nikita
child_visits = pd.read_csv(wd+"Data_Clean/summer_vs_school.csv")
child_visits.dropna(inplace=True)
adult_visits = child_visits.copy()
adult_visits["ID"] = adult_visits["ID"] - 100
visits = pd.concat([adult_visits, child_visits])
# List of columns to import
cols = ["record_id", "totalsleep_avg", "sleepefficency_avg", "waso_avg", "timeinbed_avg", "sensor_70_180", "sensor_mean",
        "sensor_55_69", "sensor_u54", "sensor_g251", "sensor_181_250", "psqi_total", "all_awakenings", "diabetes_awakenings"]
# Import data as pandas df
data = project.export_records(format_type="df", records=[str(id) for id in visits["ID"].values],
                              events=['hcl_month_3_day_1_arm_1', 'hcl_month_6_day_1_arm_1', 'hcl_month_3_day_1_arm_2', 'hcl_month_6_day_1_arm_2'], fields=cols)
data.reset_index(inplace=True)
data["redcap_event_name"].replace({'hcl_month_3_day_1_arm_1':"M3", 'hcl_month_6_day_1_arm_1':"M6",'hcl_month_3_day_1_arm_2':"M3", 'hcl_month_6_day_1_arm_2':"M6"},inplace=True)
# Drop unnecesary columns
data.drop(labels=["redcap_repeat_instrument",
          "redcap_repeat_instance"], axis=1, inplace=True)